# Conexiones e importaciones

In [1]:
import os #trabajar con el sistema operativo y variables de entorno
from dotenv import load_dotenv #cargar variable de entorno
import spotipy #librería para trabajar con la API de Spotify
from spotipy.oauth2 import SpotifyClientCredentials #autenticación con Spotify
import pandas as pd #dataframes
import requests #peticiones a APIs
import time #librería para hacer pausas dentro de las peticiones y no saturar APIs
from urllib.parse import quote #esta librería sirve para codificar las URLs (espacios y caracteres especiales -> %20, etc)
import numpy as np
import mysql.connector
from mysql.connector import Error

In [2]:
load_dotenv() #carga las variables del entorno .env; devuelve un true o false

#Spotify
mis_credenciales = SpotifyClientCredentials(
    client_id=os.getenv("SPOTIFY_CLIENT_ID"),
    client_secret=os.getenv("SPOTIFY_CLIENT_SECRET")
)
spotify = spotipy.Spotify(auth_manager=mis_credenciales)

#last fm
api_key_lastfm = os.getenv("API_KEY_LASTFM")
shared_secret_lastfm = os.getenv("SHARED_SECRET_LASTFM")

#MySQL

MYSQL_HOST = os.getenv("MYSQL_HOST")
MYSQL_USER = os.getenv("MYSQL_USER")
MYSQL_PASSWORD = os.getenv("MYSQL_PASSWORD")

# Spotify

In [3]:
generos = ["flamenco", "latin", "jazz", "rock"]
año = 2010
todas_las_canciones = []
albumes_ya_vistos = set()  # Evita duplicados: un álbum puede aparecer al buscar varios artistas (ej. colaboraciones).
datos_artistas = {}
artistas_procesados = set()

print("Buscando canciones del año", año)

for genero in generos:
    print("Género:", genero)

    # 👉 ANOTACIÓN: "genre:" SOLO funciona al buscar artistas (type="artist")
    #    No funciona con álbumes ni canciones directamente.
    resultado_artistas = spotify.search(q="genre:" + genero, type="artist", limit=50)
    artistas = resultado_artistas["artists"]["items"]

    for artista in artistas:
        nombre_artista = artista["name"]

        # 👉 ANOTACIÓN: "artist:" y "year:" SÍ se pueden usar juntos
        #    al buscar álbumes (type="album") o canciones (type="track").
        #    Aquí buscamos álbumes del artista en el año indicado.
        busqueda = "artist:" + nombre_artista + " year:" + str(año)
        resultado_albumes = spotify.search(q=busqueda, type="album", limit=50)
        albumes = resultado_albumes["albums"]["items"]
        
        for album in albumes:
            id_album = album["id"]
            nombre_album = album["name"]
            
            # Si ya vimos este álbum, saltamos al siguiente
            if id_album in albumes_ya_vistos:
                continue
            
            # Si no lo habíamos visto, lo marcamos como visto
            albumes_ya_vistos.add(id_album)

            # Pedimos las canciones del álbum (esto no usa anotaciones, es una llamada directa)
            resultado_canciones = spotify.album_tracks(id_album)
            canciones = resultado_canciones["items"]

            for cancion in canciones:
                info = {
                    "nombre": cancion["name"],
                    "artista": nombre_artista,
                    "album": nombre_album,
                    "genero": genero,
                    "año": año
                }
                todas_las_canciones.append(info)

    print("  → Canciones de este género añadidas")
    print()

# Mostramos un resumen de lo encontrado
print("RESUMEN:")

# Contamos las canciones de cada género una por una
for genero in generos:
    contador = 0
    for cancion in todas_las_canciones:
        if cancion["genero"] == genero:
            contador = contador + 1
    print(f"- {genero}: {contador} canciones")

print("Total de canciones encontradas:", len(todas_las_canciones))

todas_las_canciones_df = pd.DataFrame(todas_las_canciones)  


Buscando canciones del año 2010
Género: flamenco
  → Canciones de este género añadidas

Género: latin
  → Canciones de este género añadidas

Género: jazz
  → Canciones de este género añadidas

Género: rock
  → Canciones de este género añadidas

RESUMEN:
- flamenco: 0 canciones
- latin: 1663 canciones
- jazz: 3226 canciones
- rock: 4286 canciones
Total de canciones encontradas: 9175


In [4]:
todas_las_canciones_df

,nombre,artista,album,genero,año
0,Stand by Me,Prince Royce,Prince Royce,latin,2010
1,Corazón Sin Cara,Prince Royce,Prince Royce,latin,2010
2,Tu y Yo,Prince Royce,Prince Royce,latin,2010
3,Su Hombre Soy Yo,Prince Royce,Prince Royce,latin,2010
4,Rechazame,Prince Royce,Prince Royce,latin,2010
...,...,...,...,...,...
9170,Married In Vegas,Maná,The Chase,rock,2010
9171,Renegade,Maná,The Chase,rock,2010
9172,The Chase,Maná,The Chase,rock,2010
9173,Better Cause of You,Maná,The Chase,rock,2010


In [5]:
#todas_las_canciones_df.to_csv("canciones_2010.csv", index=False)

# Last FM

In [6]:
if not api_key_lastfm:
    print("ERROR: La variable de entorno 'API_KEY_LASTFM' no está configurada.")
else:
    print("API Key de Last.fm cargada con éxito.")

API Key de Last.fm cargada con éxito.


In [7]:
url_last_fm = ("http://ws.audioscrobbler.com/2.0/")

In [8]:
def busqueda_info_artista(nombre_artista, api_key_lastfm):
    artista_codificado = quote(nombre_artista) #esto sirve para que los espacios y caracteres especiales no interfieran con la url de la api
    
    url_last_fm = "http://ws.audioscrobbler.com/2.0/"
    params_info = {
                        'method': 'artist.getinfo',
                        'artist': artista_codificado,
                        'api_key': api_key_lastfm,
                        'format': 'json'
                    }
    try:
        response = requests.get(url_last_fm, params=params_info, timeout=10)
        response.raise_for_status()
        data = response.json()

        if "artist" in data: 
            artista_info = data['artist']
            bio_summary = artista_info.get('bio', {}).get('summary', '').split('<a href')[0].strip()
            return {
                        'bio_resumen': bio_summary,
                        'listeners': int(artista_info.get('stats', {}).get('listeners', 0)),
        }
        else:
            # Artista no encontrado por Last.fm
            return {'consulta_exitosa': False, 'error_lastfm': "No encontrado en Last.fm"}
        
    except requests.exceptions.RequestException as e:
        # Incluye HTTPError, ConnectionError, TimeoutError, etc.
        status_code = getattr(e.response, 'status_code', 'N/A')
        return {'consulta_exitosa': False, 'error_lastfm': f"Error API ({status_code}): {e}"}
    except Exception as e:
        return {'consulta_exitosa': False, 'error_lastfm': f"Error Procesamiento: {e}"}


if not api_key_lastfm:
    print("ERROR: La clave de la API de Last.fm no está configurada.")
else:
    artistas_unicos = todas_las_canciones_df['artista'].unique() #extrae los artistas sin repetir
    print(f"\nTotal de artistas únicos a consultar en Last.fm: {len(artistas_unicos)}")

    artistas_df = pd.DataFrame(artistas_unicos, columns=['artista']) #df temporal con los artistas sin repetir
    
    print("\nIniciando consultas a Last.fm...")
    
    #apply pasa el valor de la columna "artista" como primer argumento posicional. Con "args" se pasan el resto de argumentos.
    resultados_lastfm_serie = artistas_df['artista'].apply(
        busqueda_info_artista, 
        args=(api_key_lastfm,) 
    )

    # 4. Normalizar los resultados (convertir la Serie de Diccionarios a Columnas de DF)
    datos_lastfm_df = pd.json_normalize(resultados_lastfm_serie)
    datos_lastfm_df.insert(0, 'artista', artistas_unicos)
    
    print("Consultas a Last.fm terminadas y datos unidos al DataFrame.")


Total de artistas únicos a consultar en Last.fm: 83

Iniciando consultas a Last.fm...
Consultas a Last.fm terminadas y datos unidos al DataFrame.


In [9]:
datos_lastfm_df

,artista,bio_resumen,listeners
0,Prince Royce,Geoffrey Royce Rojas was born and raised in th...,703910
1,Morat,Morat is a Colombian band formed in the countr...,292207
2,Kapo,The band Kapo was formed in 2006 by Joe Muller...,127308
3,Beéle,"Brandon De Jesús López Orozco, mejor conocido ...",188781
4,Shakira,Shakira Isabel Mebarak Ripoll is a Colombian s...,4693398
...,...,...,...
78,Imagine Dragons,Imagine Dragons is an American pop band formed...,3944356
79,Maroon 5,Maroon 5 is an American pop rock band that ori...,6437494
80,Elton John,Sir Elton Hercules John (born Reginald Kenneth...,5075983
81,Juanes,Juan Esteban Aristizábal Vásquez (born August ...,1170985


In [10]:
datos_lastfm_df
datos_lastfm_df.to_csv("datos_lastfm.csv", index=False)

# Creación BD

In [11]:
try:
    cnx = mysql.connector.connect(
        host= MYSQL_HOST,
        user= MYSQL_USER,
        password= MYSQL_PASSWORD,
    )
    print('Conexión exitosa')
except Error as e:
    print('Error al conectar:', e)

try:
    mycursor = cnx.cursor()
    query = "CREATE DATABASE IF NOT EXISTS MusicStream_db"
    mycursor.execute(query)
    print("Query exitosa")
except:
    print("Error.")

Conexión exitosa
Query exitosa


In [ ]:
mycursor.execute("USE MusicStream_db")

# se añaden en varios execute porque a todas se les ha llamado query, por lo que reemplaza a la anterior y solo ejecuta la última válida

query = '''CREATE TABLE IF NOT EXISTS Canciones(
	ID_Cancion INT AUTO_INCREMENT,
    Nombre VARCHAR(70) NOT NULL,
    Artista VARCHAR(45),
    Album VARCHAR(75),
    Genero VARCHAR(30),
    Lanzamiento YEAR,
    PRIMARY KEY (ID_Cancion)
);'''
mycursor.execute(query)

query = '''CREATE TABLE IF NOT EXISTS Artistas(
    ID_Artista INT AUTO_INCREMENT,
    Artista VARCHAR(70) UNIQUE NOT NULL,
    Info VARCHAR(200),
    Oyentes INT,
    PRIMARY KEY (ID_Artista)
);'''
mycursor.execute(query)

In [13]:
# Query de inserción
mycursor.execute("USE MusicStream_db")
query_insert = """
INSERT INTO Canciones (Nombre, Artista, Album, Genero, Lanzamiento) 
VALUES (%s, %s, %s, %s, %s)
"""

df_corregido_SPOTIFY = todas_las_canciones_df.replace({np.nan: None, 'nan': None, 'NaN': None})

datos = df_corregido_SPOTIFY.values.tolist()


# Ejecutar inserción
mycursor.executemany(query_insert, datos)
cnx.commit()

print(f"{mycursor.rowcount} registros insertados")

DataError: 1406 (22001): Data too long for column 'Album' at row 51

In [ ]:
# Query de inserción
mycursor.execute("USE MusicStream_db")
query_insert = """
INSERT INTO Artistas (Artista, Info, Oyentes) 
VALUES (%s, %s, %s)
"""

df_corregido_LASTFM = datos_lastfm_df.replace({np.nan: None, 'nan': None, 'NaN': None})

datos = df_corregido_LASTFM.values.tolist()


# Ejecutar inserción
mycursor.executemany(query_insert, datos)
cnx.commit()

print(f"{mycursor.rowcount} registros insertados")

print(f"{mycursor.rowcount} registros insertados")